# Model search

## Getting started

This script will go through assembling the main components of a complete pipeline for counting seals in high-resolution satellite imagery. 

<img src="jupyter_notebook_images/pipeline.png">
** *environmental covariates not currently incorporated **

If you followed the *training_set_generation* jupyter notebook (also present in this repo), you should have training sets generated and hyperparameter sets to try out, and be ready to search for a best performing model setup. Models will be evaluated at the three levels of the detection pipeline: **1)** their ability to identify scenes with seals, **2)** retrieve seal haul outs **3)** and count individual seals. Before training and validating model/hyperparameter combinations, we need to load the required python modules. Running this script will also display a list of training classes. 


In [1]:
# import required packages
import os
import rasterio
import pandas as pd
import numpy as np
from PIL import Image 
import matplotlib.pyplot as plt
import matplotlib as mpl
from functools import reduce
from model_library import * 

%matplotlib inline
mpl.rcParams['figure.dpi']= 400


# display class names
class_names = sorted([subdir for subdir in os.listdir('./training_sets/training_set_vanilla/training')])
print(class_names)

['crabeater', 'crack', 'emperor', 'glacier', 'ice-sheet', 'marching-emperor', 'open-water', 'other', 'pack-ice', 'rock', 'weddell']


## Visualizing training images (Optional)

To get a better sense for what the training set is like, the next cell will display a few random images from the training classes. Displayed images are extracted from a pool of ~70000 training images. 

In [ ]:
# store images
images = []

# loop over labels
for label in class_names:
    for path, _, files in os.walk('./training_sets/training_set_vanilla/training/{}'.format(label)):
        files = np.random.choice(files, 5)
        for filename in files:
            images.append(np.asarray(Image.open(os.path.join(path, filename))))

images = np.array(images)

# display images 
ncols=len(class_names)
nindex, height, width, intensity = images.shape
nrows = nindex//ncols
assert nindex == nrows*ncols
result = (images.reshape(nrows, ncols, height, width, intensity)
          .swapaxes(1,2)
          .reshape(height*nrows, width*ncols, intensity))

plt.imshow(result)
cur_axes = plt.gca()
cur_axes.axes.get_xaxis().set_visible(False)
cur_axes.axes.get_yaxis().set_visible(False)
plt.show()
    


## Training - haulout detector

The first step to find a best performing model is to train different model setups using our training set. To keep track of which combinations we have tried and the specifics of each model setup, we will create a pandas DataFrame with model definitions.

In [2]:
# generate model combinations
combinations_haul = {'model_architecture': ['Resnet18'] * 4 + ['NasnetA'] * 4,
                     'training_dir': ['training_set_vanilla', 'training_set_multiscale_A'] * 4,
                     'hyperparameter_set': ['A'] * 4 + ['B'] * 4,
                     'cv_weights': ['NO', 'NO', 'A', 'A'] * 2,
                     'output_name': ['model{}'.format(i) for i in range(1,9)]}

combinations_haul = pd.DataFrame(combinations_haul)

# create folders for resulting files
for mdl in combinations_haul['output_name']:
    if not os.path.exists("./saved_models/haulout/{}".format(mdl)):
        os.makedirs("./saved_models/haulout/{}".format(mdl)) 


We can then provide model combinations created above as arguments to the training script, *train_sealnet.py*. A list of required arguments can be displayed by running the cell below.

In [ ]:
%run train_sealnet.py -h

In [ ]:
# iterate over combinations
for row in combinations_haul.iterrows():
    t_dir, arch, hyp_st, cv_wgt, out = row[1]['training_dir'], row[1]['model_architecture'], row[1]['hyperparameter_set'], row[1]['cv_weights'], row[1]['output_name']
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('./saved_models/haulout/{}/'.format(out)): 
        print('{} was already trained'.format(out))
        continue
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet.py --training_dir=$t_dir --model_architecture=$arch --hyperparameter_set=$hyp_st --cv_weights=$cv_wgt --output_name=$out
    
      

## Training - single seal detector



Similar to training a model at the haulout level, we start by training a number of model combinations. To keep track of which combinations we have tried and the specifics of each model setup, we will create a pandas DataFrame with model definitions.

In [16]:
# generate model combinations
combinations_single = {'model_architecture': ['WideResnetA'] * 4,
                       'training_dir': ['training_set_vanilla', 'training_set_multiscale_A'] * 2,
                       'hyperparameter_set': ['C'] * 4,
                       'cv_weights': ['NO', 'NO', 'A', 'A'],
                       'output_name': ['model{}'.format(i) for i in range(1,5)]}

combinations_single = pd.DataFrame(combinations_single)

# create folders for resulting files
for mdl in combinations_single['output_name']:
    if not os.path.exists("./saved_models/single_seal/{}".format(mdl)):
        os.makedirs("./saved_models/single_seal/{}".format(mdl)) 

Similar to the haulout models, we can provide model combinations created above as arguments to the training script, *train_sealnet.py*. 

In [17]:
# iterate over combinations
for row in combinations_single.iterrows():
    t_dir, arch, hyp_st, cv_wgt, out = row[1]['training_dir'], row[1]['model_architecture'], row[1]['hyperparameter_set'], row[1]['cv_weights'], row[1]['output_name']
    # check if model is already trained
    if "{}.tar".format(out) in os.listdir('./saved_models/single_seal/{}/'.format(out)): 
        print('{} was already trained'.format(out))
        continue
    
    print()
    !echo training $out
    print()
    
    # run training
    !python train_sealnet.py --training_dir=$t_dir --model_architecture=$arch --hyperparameter_set=$hyp_st --cv_weights=$cv_wgt --output_name=$out
    


training model1

Epoch 1/30
----------
training Loss: 0.0616 Acc: 0.5316
validation Loss: 0.3756 Acc: 0.6643
training time: 0.0h 1m 23s

Epoch 2/30
----------
training Loss: 0.0499 Acc: 0.6131
validation Loss: 0.3693 Acc: 0.6852
training time: 0.0h 2m 46s

Epoch 3/30
----------
training Loss: 0.0463 Acc: 0.6395
validation Loss: 0.3615 Acc: 0.6668
training time: 0.0h 4m 9s

Epoch 4/30
----------
training Loss: 0.0448 Acc: 0.6495
validation Loss: 0.3048 Acc: 0.7343
training time: 0.0h 5m 33s

Epoch 5/30
----------
training Loss: 0.0433 Acc: 0.6628
validation Loss: 0.3437 Acc: 0.6872
training time: 0.0h 6m 57s

Epoch 6/30
----------
training Loss: 0.0416 Acc: 0.6740
validation Loss: 0.3373 Acc: 0.6710
training time: 0.0h 8m 20s

Epoch 7/30
----------
training Loss: 0.0411 Acc: 0.6803
validation Loss: 0.3202 Acc: 0.6830
training time: 0.0h 9m 43s

Epoch 8/30
----------
training Loss: 0.0401 Acc: 0.6873
validation Loss: 0.3305 Acc: 0.6888
training time: 0.0h 11m 8s

Epoch 9/30
----------
t

training Loss: 0.0308 Acc: 0.6618
validation Loss: 0.3233 Acc: 0.7151
training time: 0.0h 15m 14s

Epoch 9/30
----------
training Loss: 0.0301 Acc: 0.6694
validation Loss: 0.3263 Acc: 0.7263
training time: 0.0h 16m 37s

Epoch 10/30
----------
training Loss: 0.0294 Acc: 0.6741
validation Loss: 0.3228 Acc: 0.7014
training time: 0.0h 18m 3s

Epoch 11/30
----------
training Loss: 0.0291 Acc: 0.6782
validation Loss: 0.2781 Acc: 0.7544
training time: 0.0h 19m 28s

Epoch 12/30
----------
training Loss: 0.0288 Acc: 0.6818
validation Loss: 0.2853 Acc: 0.7460
training time: 0.0h 20m 52s

Epoch 13/30
----------
training Loss: 0.0278 Acc: 0.6896
validation Loss: 0.2957 Acc: 0.7442
training time: 0.0h 22m 16s

Epoch 14/30
----------
training Loss: 0.0277 Acc: 0.6911
validation Loss: 0.3169 Acc: 0.6914
training time: 0.0h 23m 39s

Epoch 15/30
----------
training Loss: 0.0273 Acc: 0.6934
validation Loss: 0.2971 Acc: 0.7174
training time: 0.0h 25m 3s

Epoch 16/30
----------
training Loss: 0.0274 Acc: 


Epoch 1/30
----------
Traceback (most recent call last):
  File "train_sealnet.py", line 245, in <module>
    main()
  File "train_sealnet.py", line 241, in main
    num_epochs=hyperparameters[args.hyperparameter_set]['epochs'])
  File "train_sealnet.py", line 158, in train_model
    outputs = model(inputs)
  File "/home/bento/anaconda3/lib/python3.6/site-packages/torch/nn/modules/module.py", line 491, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/bento/PycharmProjects/Seals_branches/Master/Seals/custom_architectures/wide_resnet.py", line 92, in forward
    out = F.avg_pool2d(out, 7)
RuntimeError: Given input size: (64x4x4). Calculated output size: (64x0x0). Output size is too small at /opt/conda/conda-bld/pytorch_1524590031827/work/aten/src/THCUNN/generic/SpatialAveragePooling.cu:63

training model8

Epoch 1/30
----------
Traceback (most recent call last):
  File "train_sealnet.py", line 245, in <module>
    main()
  File "train_sealnet.py", line 241, in main


## Validation - haulout level

We can now load the models we just trained to get measurements of precision and recall for all positive classes. For every model combination we trained, *validate_sealnet.py* will run a full validation round and write given label/correct label pairs to a .csv file. The resulting .csv file is then imported by an R script, *plot_confusion_matrix.R*, which saves a confusion matrix figure and a .csv spreadsheet with precision and recall for all classes of interest. 

In [ ]:
# DataFrame to combine all metrics 
comb_prec_recall = pd.DataFrame()

# iterate over trained models
for row in combinations_haul.iterrows():
    
    # read hyperparameters
    t_dir, arch, hyp_st, out = row[1]['training_dir'], row[1]['model_architecture'], row[1]['hyperparameter_set'], row[1]['output_name']
    
    # check if model file is available
    if "{}.tar".format(out) not in os.listdir('./saved_models/haulout/{}/'.format(out)): 
        continue
        
    else:
        print()
        !echo validating $out
        print()
        
        # run validation
        !python validate_sealnet_haulout.py --training_dir=$t_dir --model_architecture=$arch --hyperparameter_set=$hyp_st --model_name=$out
        
        # extract performance
        !Rscript plot_confusion_matrix.R $out
        
        # accumulate performance scores
        comb_prec_recall = comb_prec_recall.append(pd.read_csv('./saved_models/haulout/{}/{}_haul_prec_recall.csv'.format(out, out)))
    
    
# Write combined metrics to csv
comb_prec_recall.to_csv('./saved_models/haulout/pooled_haul_prec_recall.csv')

# Plot combined metrics
!Rscript plot_comparison.R
    

## Validation - scene level

The first step to validate models at the scene level is to tile out rasters to model input sizes. The following cell will search for all rasters in a dir, check if they are present in the scene_bank (see  *training_set_generation.ipynb*), store an affine matrix for that scene to go from the tile's index in the scene to projected coordinates and create tiles with the correct dimensions(including multi-scale models) for all model architectures in the 'combinations' DataFrame. 

In [ ]:
# load scene bank
scene_bank = pd.read_csv('./training_sets/seals_scene_bank.csv')

# point to raster dir
raster_dir = '/home/bento/imagery'

# create output folder
%mkdir './tiled_images'

# store data transforms
affine_transforms = {}

# loop through rasters creating tiled versions
print('\nTiling rasters:')
for path, _, files in os.walk(raster_dir):
    # filter rasters to include only ones present in the scene bank
    files = [file for file in files if file in pd.unique(scene_bank['scene'])]
    for count, filename in enumerate(files):
        filename_lower = filename.lower()
        input_path = (os.path.join(path, filename))
        # extract data transform to go from raster index to ESPG3031 coordinates
        with rasterio.open(input_path) as src:
            affine_transforms[filename] = [src.transform[1], src.transform[2], src.transform[0], src.transform[4], src.transform[5], src.transform[3]]
        # loop over models
        for row in combinations_haul.iterrows():
            # get model input size
            input_size = model_archs[row[1]['model_architecture']]
            ts_scales = training_sets[row[1]['training_dir']]['scale_bands']
            scales = [str(int(input_size * scale / ts_scales[0])) for scale in ts_scales]
            scale_bands = reduce(lambda x,y: x + '_' + y, scales)
            output_folder = './tiled_images/{}/{}'.format(input_path, scale_bands)
            # create a tiled out image for that model if it doesn't exist
            if os.path.exists(output_folder):
                print('{} is already tiled'.format(filename))
                continue
            !python tile_raster.py --scale_bands=$scale_bands  --input_image=$input_path --output_folder='./tiled_images'
        print('\nProcessed {} out of {} rasters'.format(count + 1, len(files)))

# write data transforms to csv
affine_transforms = pd.DataFrame(affine_transforms)
affine_transforms.to_csv('affine_transforms.csv')

With rasters tiled to the correct dimensions we can now run validation at the scene level. This procedure will work as follows: for each model, classify all images tiled in the previous step and save classification results to a pandas DataFrame. Results will be compared with the scene_bank to measure precision and recall at detecting which scenes contain positive entries (i.e. seals, penguins or even both, depending on the scene_bank being used). Locations marked as having seal haulouts will be used to detect individual seals and validate models at the seal level. To get validation results, run the following cell: 

In [8]:
# select scene bank
bank = "seals_scene_bank.csv" # for penguins: bank = penguins_scene_bank.csv

# load scene bank
scene_bank = pd.read_csv('./training_sets/{}'.format(bank))
positive_classes = [label for label in scene_bank.columns if label in class_names]

# store precision and recall
comb_prec_rec = pd.DataFrame()

# loop through models
for row in combinations_haul.iterrows():
    # store model classifications to get seal locations for seal level validation
    val_scene_model = pd.DataFrame()
    
    # get model settings
    arch = row[1]['model_architecture']
    model_name = row[1]['output_name']
    t_dir = row[1]['training_dir']
    hyp_st = row[1]['hyperparameter_set']
    
    # check if model was already processed
    if "{}_scene_val.csv".format(model_name) in os.listdir('./saved_models/haulout/{}/'.format(model_name)): 
        continue
    
    # get scale bands
    input_size = model_archs[arch]['input_size']
    ts_scales = training_sets[t_dir]['scale_bands']
    scales = [str(int(input_size * scale / ts_scales[0])) for scale in ts_scales]
    scale_bands = reduce(lambda x,y: x + '_' + y, scales)
    
    # count true positives, true negatives, false positives, false negatives
    model_stats = {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0}
    
    # loop through tiled rasters
    scenes = os.listdir('./tiled_images')
    print('\nClassifying tiled scenes with {}:'.format(model_name))
    for count, scene in enumerate(scenes):
        input_folder = './tiled_images/{}/{}'.format(scene, scale_bands)
        %run predict_sealnet_batches.py --training_dir=$t_dir --hyperparameter_set=$hyp_st --model_architecture=$arch --model_name=$model_name --data_dir=$input_folder --affine_transforms='affine_transforms.csv' --scene=$scene        
        val_scene_model = val_scene_model.append(pd.read_csv('./saved_models/haulout/{}/{}_scene_val_tmp.csv'.format(model_name, model_name)), ignore_index=True)
        # record if scene is TP, TN, FP or FN
        detected = False
        for label in positive_classes:
            if label in val_scene_model['label']:
                detected = True
        # if scene is occupied
        if int(scene_bank.loc[scene_bank['scene'] == scene]['present']):
            if detected:
                model_stats['TP'] += 1
            else:
                model_stats['FN'] += 1
        # if scene is empty
        else:
            if detected:
                model_stats['FP'] += 1
            else:
                model_stats['TN'] += 1
    
        print('\nClassified {} out of {} tiled scenes'.format(count + 1, len(scenes)))
    
    # record precision and recall for model
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    comb_prec_rec.append({'model': model_name, 'precision': precision, 'recall': recall})
    
    # save final classifications to csv and cleanup temporary stats
    val_scene_model.to_csv('./saved_models/haulout/{}/{}_scene_val.csv'.format(model_name, model_name))
    !rm $('./saved_models/haulout/{}/{}_scene_val_tmp.csv'.format(model_name, model_name))
    
# save combined precision recall and combine classifications
comb_prec_rec.to_csv('./saved_models/haulout/pooled_scene_prec_recall.csv')
    

A

Classifying tiled scenes with model1:

Classified 1 out of 35 tiled scenes


RuntimeError: Found 0 files in subfolders of: ./tiled_images/WV03_20151030210819_104001001351DB00_15OCT30210819-P1BS-500658695070_01_P001_u08rf3031.tif/224_224_224
Supported extensions are: .jpg,.jpeg,.png,.ppm,.bmp,.pgm,.tif


Classified 2 out of 35 tiled scenes


RuntimeError: Found 0 files in subfolders of: ./tiled_images/WV03_20151022204340_10400100127E2F00_15OCT22204340-P1BS-500652476030_01_P001_u08rf3031.tif/224_224_224
Supported extensions are: .jpg,.jpeg,.png,.ppm,.bmp,.pgm,.tif


Classified 3 out of 35 tiled scenes


...


Classified 25 out of 35 tiled scenes


RuntimeError: Found 0 files in subfolders of: ./tiled_images/WV03_20151021104725_10400100124A0000_15OCT21104725-P1BS-500652455030_01_P001_u08rf3031.tif/224_224_224
Supported extensions are: .jpg,.jpeg,.png,.ppm,.bmp,.pgm,.tif


Classified 26 out of 35 tiled scenes


FileNotFoundError: [Errno 2] No such file or directory: './tiled_images/move_files.sh/224_224_224'

TypeError: cannot convert the series to <class 'int'>

## Validation -- single seal level

In [ ]:
%mkdir classified_images
for x in class_names:
    dir = './classified_images/' + x
    %mkdir -v $dir
    
%run predict_sealnet.py "training_set_vanilla" "NasnetA" "model5" "./to_classify"